In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [2]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [3]:
Y_train

array([8, 4, 1, ..., 6, 1, 0], dtype=int64)

In [4]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[9 4 8 ... 2 2 9] [8 4 1 ... 6 1 0]
0.11282926829268293
Iteration:  10
[5 4 8 ... 6 2 6] [8 4 1 ... 6 1 0]
0.24241463414634146
Iteration:  20
[5 4 8 ... 6 2 6] [8 4 1 ... 6 1 0]
0.33602439024390246
Iteration:  30
[5 4 1 ... 6 2 6] [8 4 1 ... 6 1 0]
0.3927560975609756
Iteration:  40
[0 4 1 ... 6 1 6] [8 4 1 ... 6 1 0]
0.44909756097560977
Iteration:  50
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.504219512195122
Iteration:  60
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.5507560975609757
Iteration:  70
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.5869024390243902
Iteration:  80
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.618219512195122
Iteration:  90
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.6416585365853659
Iteration:  100
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.6630731707317074
Iteration:  110
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.6826585365853659
Iteration:  120
[0 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.6987560975609756
Iteration:  130
[8 4 1 ... 6 1 0] [8 4 1 ... 6 1 0]
0.7115853658536585
Iteration:  140

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [8]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[7 9 8 9 1 6 3 7 1 3 0 8 9 1 9 9 6 2 6 3 7 1 1 5 6 5 5 9 7 4 8 6 3 9 0 1 9
 4 8 4 4 7 7 6 5 9 8 3 0 7 8 6 8 0 7 6 9 8 5 6 8 8 2 2 9 1 7 8 9 7 9 2 4 6
 7 9 4 9 1 2 0 1 8 0 7 8 0 3 7 5 4 4 4 9 1 5 4 3 6 2 5 1 3 0 0 7 2 7 6 2 7
 3 1 7 5 0 4 8 5 4 1 1 9 3 6 7 1 5 7 2 0 7 2 1 7 2 9 5 0 3 2 6 7 8 1 3 2 8
 3 2 6 0 6 9 9 2 9 5 4 5 8 7 7 3 9 2 9 6 0 3 3 0 7 0 8 1 9 9 9 6 0 2 7 1 2
 2 2 3 8 8 6 6 0 6 8 4 4 2 6 2 1 0 1 5 3 8 0 7 2 8 6 3 1 8 7 4 3 0 6 0 2 7
 2 9 2 7 0 7 6 3 6 5 0 7 3 9 8 4 4 0 4 1 9 6 5 6 2 4 0 8 2 6 4 0 7 6 5 5 1
 0 2 1 7 5 6 3 1 8 7 3 5 9 3 4 6 4 4 6 9 7 0 2 6 6 0 5 2 9 9 2 1 3 9 7 3 8
 7 1 2 0 1 3 9 5 1 8 5 9 1 6 3 6 9 3 0 0 0 6 8 4 7 5 4 7 9 3 3 9 7 9 9 0 9
 0 7 5 3 6 9 3 0 6 8 8 2 8 6 6 4 3 0 5 9 0 7 2 5 8 1 6 7 0 2 1 1 6 9 8 8 1
 3 0 6 9 1 8 8 2 7 1 6 3 0 6 8 4 9 8 5 8 8 8 7 9 7 4 3 4 5 9 4 8 0 1 7 1 8
 7 8 5 0 5 1 3 1 1 6 4 0 3 6 4 6 1 1 9 7 6 6 5 9 0 3 1 2 5 5 1 1 8 0 1 2 8
 3 8 7 2 7 0 5 9 9 3 0 1 5 9 6 0 7 6 8 5 2 9 3 3 4 6 1 8 7 3 0 5 4 2 3 2 2
 0 4 0 6 3 1 8 1 2 9 4 0 

0.849